Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://wp.pl")
ed.links

['https://www.wp.pl',
 'https://poczta.wp.pl',
 'https://pilot.wp.pl/tv?utm_source=stronaglowna_wp&utm_medium=belka_sg&utm_campaign=wppilot',
 'https://www.money.pl/',
 'https://finanse.wp.pl/',
 'https://www.money.pl/gielda/',
 'https://www.money.pl/pieniadze/nbp/srednie/',
 'https://www.money.pl/rejestr-firm/',
 'https://prawo.money.pl/',
 'https://www.money.pl/pieniadze/kalkulator/',
 'https://www.money.pl/sekcja/ppk/',
 'https://www.money.pl/podatki/kalkulatory/plac/',
 'https://www.money.pl/emerytury/',
 'https://www.money.pl/banki/elixir/',
 'https://praca.money.pl/',
 'https://sportowefakty.wp.pl/',
 'https://sportowefakty.wp.pl/zuzel',
 'https://sportowefakty.wp.pl/zuzel/pge-ekstraliga',
 'https://sportowefakty.wp.pl/pilka-nozna',
 'https://sportowefakty.wp.pl//pilka-nozna/pko-ekstraklasa',
 'https://sportowefakty.wp.pl/siatkowka',
 'https://sportowefakty.wp.pl/skoki-narciarskie',
 'https://sportowefakty.wp.pl/formula-1',
 'https://sportowefakty.wp.pl/koszykowka',
 'https://spo

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://wp.pl - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://www.wp.pl
https://poczta.wp.pl
https://pilot.wp.pl/tv?utm_source=stronaglowna_wp&utm_medium=belka_sg&utm_campaign=wppilot
https://www.money.pl/
https://finanse.wp.pl/
https://www.money.pl/gielda/
https://www.money.pl/pieniadze/nbp/srednie/
https://www.money.pl/rejestr-firm/
https://prawo.money.pl/
https://www.money.pl/pieniadze/kalkulator/
https://www.money.pl/sekcja/ppk/
https://www.money.pl/podatki/kalkulatory/plac/
https://www.money.pl/emerytury/
https://www.money.pl/banki/elixir/
https://praca.money.pl/
https://sportowefakty.wp.pl/
https://sportowefakty.wp.pl/zuzel
https://sportowefakty.wp.pl/zuzel/pge-ekstraliga
https://sportowefakty.wp.pl/pilka-nozna
https://sportowefakty.wp.pl//pilka-nozna

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/meta-llama/Llama-4-Scout-17B-16E-Instruct',
 '/Qwen/Qwen2.5-Omni-7B',
 '/deepseek-ai/DeepSeek-V3-0324',
 '/all-hands/openhands-lm-32b-v0.1',
 '/openfree/flux-chatgpt-ghibli-lora',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/jamesliu1217/EasyControl_Ghibli',
 '/spaces/VAST-AI/TripoSG',
 '/spaces/Stable-X/Hi3DGen',
 '/spaces/ByteDance/InfiniteYou-FLUX',
 '/spaces',
 '/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset-v1',
 '/datasets/open-thoughts/OpenThoughts2-1M',
 '/datasets/virtuoussy/Multi-subject-RLVR',
 '/datasets/glaiveai/reasoning-v1-20m',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/g

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'home page', 'url': 'https://huggingface.co/'},
  {'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI 

In [24]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussions', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmeta-llama/Llama-4-Scout-17B-16E-Instruct\nUpdated\nabout 4 hours ago\n•\n16k\n•\n386\nQwen/Qwen2.5-Omni-7B\nUpdated\n6 days ago\n•\n90k\n•\n1.21k\ndeepseek-ai/DeepSeek-V3-0324\nUpdated\n11 days ago\n•\n158k\n•\n2.37k\nall-hands/openhands-lm-32b-v0.1\nUpdated\n3 days ago\n•\n3.62k\n•\n277\nopenfree/flux-chatgpt-ghibli-lora\nUpdated\n1 day ago\n•\n6.7k\n•\n278\nBrowse 1M+ models\nSpaces\nRunnin

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

---

## Welcome to Hugging Face 🌟

**“The AI community building the future.”**  
Hugging Face is at the forefront of advancing machine learning (ML) and artificial intelligence (AI) through collaboration, innovation, and community engagement. Our platform allows users to explore, share, and utilize over **1 million models** and **250,000 datasets**.

---

## Our Offerings

### **Models & Datasets**
- **Explore AI Models:** Discover trending models like meta-llama and Qwen, curated for high-performance AI applications.
- **Datasets:** Access a variety of datasets, from open thoughts to specialized medical reasoning datasets, facilitating extensive research and application development.

### **Spaces**
- Join in the excitement of our **Spaces**, where applications are built collaboratively. Currently, we host thousands of applications including DeepSite, TripoSG, and InfiniteYou.

### **Compute & Enterprise Solutions**
- Accelerate your AI projects with our paid compute offerings starting at **$0.60/hour** for GPU usage. 
- Our **Enterprise solutions** provide advanced security features and dedicated support for teams at **$20/user/month**.

---

## Who We Serve

Over **50,000 organizations**, ranging from startups to giants like Google and Microsoft, utilize Hugging Face to build and enhance their AI capabilities. Our community-driven approach makes Hugging Face a preferred partner for enterprises wanting to leverage the latest in machine learning technology.

- **Highlighted Clients:**  
  - AI at Meta  
  - Amazon  
  - Grammarly

---

## Company Culture

At Hugging Face, we foster a vibrant and inclusive community of machine learning enthusiasts and experts. Our values are centered around:
- **Collaboration:** We encourage sharing and networking among our users and contributors.
- **Innovation:** Continuous improvement and learning are at the core of our operations.
- **Transparency:** Open-source practices underpin our commitment to building trust within the community.

Our team is growing, and we actively seek passionate individuals ready to innovate in AI. Join us to be part of a movement that shapes the future!

---

## Careers at Hugging Face

Looking for a challenging and rewarding career? At Hugging Face, we offer numerous opportunities for developers, researchers, and AI enthusiasts.  
Explore our current job openings and find a role that suits your skills and interests!

**Explore Careers:** [Join Our Team!](https://huggingface.co/jobs)

---

## Join Us

Whether you are a developer seeking to build the next big ML application, a researcher looking for comprehensive datasets, or an organization aiming to integrate AI into your operations, Hugging Face is here to empower you. 

**Get Started Today!**  
Visit us at [Hugging Face](https://huggingface.co), or connect with us on our social platforms:

- [GitHub](https://github.com/huggingface)
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://www.linkedin.com/company/huggingface)
- [Discord](https://discord.com/invite/huggingface)

---

Let’s accelerate the future of machine learning together!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Hugging Face Brochure

## Welcome to Hugging Face

### The AI Community Building the Future

At Hugging Face, we are at the forefront of the artificial intelligence revolution. Our platform empowers the machine learning community to collaborate and innovate. With over **1 million models**, **250k datasets**, and **400k applications**, we are paving the way for ML enthusiasts and professionals alike.

---

## Our Offerings

- **Models**: Explore and utilize an extensive library of pre-trained models ranging from natural language processing to image generation.
- **Datasets**: Access a wealth of datasets for diverse ML tasks tailored to both beginners and researchers.
- **Spaces**: Collaborate on and showcase your ML applications with ease, facilitating a vibrant development community.
- **Enterprise Solutions**: We offer robust compute solutions and enterprise-grade security, tailored for businesses to build and deploy AI with confidence.

### Featured Models
- **meta-llama/Llama-4-Scout-17B-16E-Instruct**
- **Qwen/Qwen2.5-Omni-7B**
- **deepseek-ai/DeepSeek-V3-0324**
  
Explore our trending and new releases, updated regularly to keep up with the latest in AI advancements.

---

## Who We Serve

More than **50,000 organizations** trust Hugging Face, including notable names like:
- Google
- Microsoft
- Amazon
- Intel
- Grammarly

From startups to enterprise giants, our diverse clientele benefits from state-of-the-art AI solutions, enhancing their products and services with the power of machine learning.

---

## Company Culture

At Hugging Face, our culture thrives on collaboration, creativity, and innovation. We believe in building a strong community where ideas flow freely and everyone feels empowered to contribute. Our open-source ethos lays the foundation for teamwork, and we regularly engage with our users to improve our offerings.

### Join Us!

We are always on the lookout for passionate individuals to join our growing team. If you are excited about AI and want to make a significant impact in the tech world, explore our careers page to find roles that suit your skills and aspirations. 

**Current Openings**: 
- Software Engineers
- Data Scientists
- Community Managers

Embrace the future of AI with Hugging Face – where innovation meets collaboration.

---

## Connect with Us

Stay updated and join our vibrant community:
- [GitHub](https://github.com/huggingface)
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://www.linkedin.com/company/huggingface)
- [Discord](https://discord.com/invite/huggingface)

For more information, visit our website: [huggingface.co](https://huggingface.co)

---

This  brochure provides a comprehensive overview of Hugging Face, emphasizing their offerings, customer base, company culture, and opportunities for prospective recruits.

In [21]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}


# Anthropic Brochure

## Who We Are
Anthropic is a pioneering AI safety and research company based in San Francisco, committed to building reliable, interpretable, and steerable AI systems. We believe that AI will significantly impact the world, and our mission is to ensure that these systems are safe and beneficial for everyone.

## Our Purpose
At Anthropic, our purpose is clear: to create AI systems that people can rely on. We engage in frontier research to understand and address the opportunities and risks associated with AI technologies, advocating for systemic safety in the industry.

## Our Products
Introducing **Claude**, our advanced AI model family, with the latest iteration being **Claude 3.7 Sonnet**, recognized for its enhanced intelligence and hybrid reasoning capabilities. We also provide Claude Code, an innovative tool that supports coding tasks efficiently.

### Product Offerings:
- **Claude for Enterprise**: AI solutions tailored for business needs.
- **Claude API**: Build custom, AI-powered applications and experiences.
- **Enterprise and Education Plans**: Tailored offerings for organizations and educational institutions.

## Company Culture
Anthropic fosters an interdisciplinary and collaborative environment composed of researchers, engineers, policy experts, and operational leaders. Our team, with diverse backgrounds from various sectors such as NASA and the armed forces, unites to cultivate innovative AI technologies. 

### Core Values:
- **Safety First**: We regard AI safety as a scientific discipline and consistently strive to apply our research for reliable outcomes.
- **Collaborative Spirit**: We believe in partnership, working closely with civil society, government, academia, and industry stakeholders to ensure safety across the ecosystem.
- **Transparency**: We regularly share insights from our research, promoting an open dialogue around AI's impacts.

## Careers at Anthropic
Join our dynamic team! We offer a variety of roles for individuals passionate about advancing AI safety. If you're driven by purpose and innovation, explore our open positions across research, engineering, and operations.

### Why Work with Us:
- Engage in meaningful and impactful projects.
- Collaborate with a diverse team of experts from various disciplines.
- Contribute to frontline AI research and development with a focus on safety.

## Our Commitments
At Anthropic, we take our responsibilities seriously. Our commitments to transparency, responsible scaling, security, and compliance are foundational to our operations and guide our strategic decisions.

## Get in Touch
Explore our products, research insights, and career opportunities on our website: [Anthropic.ai](https://www.anthropic.ai).

---

**Anthropic: Building AI you can trust, for a better future.**

In [23]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company profile', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Welcome to Hugging Face: Where AI Hugs You Back!

**Your Friendly AI Gurus**  
At Hugging Face, we're not just building the future; we're hugging it too! Join our vibrant community, where the machine learning folks meet for coffee, model collaborations, and some good ol’ data-sharing giggles. Whether you’re a seasoned ML wizard or just an enthusiastic apprentice, prepare to be amazed!

---

## Models, Datasets, and Spaces – Oh My!

- **1M+ Models**: Choose one (or several) to generate your next winning AI application! From meta-llama to the quirky Ghibli EasyControl, we’ve got models galore. Who knew hugging could be this powerful?
- **250k+ Datasets**: Our proprietary database is so spacious, it could fit an elephant! (Not that we recommend it.)
- **400k+ Applications**: From generating images to crafting the next Oscar-winning 3D film scripts, the possibilities are as infinite as our spaces. 

*Note: We are not responsible for any films that may star AI-generated llamas. They're notoriously difficult to direct.*

---

## Culture Shock: Hugging Face Style

Imagine a workplace where brainstorming sessions involve virtual hugs and machine learning magic. Here you get to work with:

- 🚀 **Innovators**: Our community includes big names like Google, Amazon, and Microsoft. It’s essentially like a celebrity meetup, minus the paparazzi.
- 🌈 **Collaborators**: We embrace open-source like a long-lost sibling! We work together and share ideas because sharing is caring...or at least it's less lonely!
- 💼 **Career Growth**: Stuck in a rut? Join us! At Hugging Face, you can build your portfolio and grow your profile while chatting about the latest in AI over coffee or cookies. 

---

## Join the Hugging Family!

Looking for a career that will make you burst with excitement? We’re hiring! Check out our current openings where you’ll get to work on:

- **Transformers**: Not the ones that change into trucks! Think Neural Networks that make sense of language.
- **Diffusers**: No, not the fancy car mods, but tools that create stunning visuals!
- **Smol Agents**: Tiny but mighty! (That’s just how we like our assistants).

---

## A Community Like No Other

At Hugging Face, our motto is simple: If you’re not hugging, you’re not living! 

Whether you’re a budding student, a data scientist, a seasoned investor, or just someone who loves warm, fuzzy AI models, you belong here. 

💌 **Join us today** - because nothing says "I love AI" like a good old-fashioned hug!

[Visit Us!](https://huggingface.co)

---

*Disclaimer: Hugging Face cannot be held responsible for any excessive hugging, AI-related excitement, or spontaneous laughter that may occur while browsing our site.*